In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

import mpisppy.utils.sputils as sputils
from mpisppy.opt.lshaped import LShapedMethod

from utils.functions import extract_tem_min, get_data_fromNSRDB, power_PV_calculation, calculate_WT_power, tabbed_df_viewer, plot_time_series
from utils.model import create_model

from utils.maestros import Maestro_Optimizacion

[    0.00] Initializing mpi-sppy


## Dimensionar MR OffGrid con escenarios de irradiancia y velocidad del viento

### Seleccionar ubicación

In [2]:
location = {"lat":4.433, "lon":-69.8, "name":"Cumaribo", "name_data" : "cumaribo", "year_deterministic": 2020}
#location = {"lat":12.583, "lon":-81.706, "name":"San Andrés"}

### Escenario determinista

In [3]:
df_meteo, info = get_data_fromNSRDB(location["lat"], location["lon"], location["year_deterministic"])
date_vec = np.vectorize(datetime)
df_index = date_vec(df_meteo.Year.values,df_meteo.Month.values,df_meteo.Day.values, df_meteo.Hour.values, df_meteo.Minute.values, tzinfo=None)
df_meteo.index = df_index
df_meteo

,Year,Month,Day,Hour,Minute,GHI,DHI,DNI,Wind Speed,Temperature,Solar Zenith Angle
2020-01-01 00:00:00,2020,1,1,0,0,0,0,0,1.0,22.8,160.94
2020-01-01 01:00:00,2020,1,1,1,0,0,0,0,1.0,22.5,153.62
2020-01-01 02:00:00,2020,1,1,2,0,0,0,0,1.1,22.2,141.96
2020-01-01 03:00:00,2020,1,1,3,0,0,0,0,1.1,22.0,128.89
2020-01-01 04:00:00,2020,1,1,4,0,0,0,0,1.2,21.8,115.29
...,...,...,...,...,...,...,...,...,...,...,...
2020-12-31 19:00:00,2020,12,31,19,0,0,0,0,1.0,24.6,109.60
2020-12-31 20:00:00,2020,12,31,20,0,0,0,0,0.9,24.2,123.29
2020-12-31 21:00:00,2020,12,31,21,0,0,0,0,0.9,23.8,136.64
2020-12-31 22:00:00,2020,12,31,22,0,0,0,0,0.9,23.4,149.07


### Catálogo de equipos

In [4]:
catalogo = pd.read_excel("Catalogo.xlsx", sheet_name=None, index_col = 0)
tabbed_df_viewer(catalogo)

### Series temporales

In [5]:
series_data = pd.read_excel("CaseData.xlsx", sheet_name = f"series_{location['name_data']}")

plot_time_series(series_data,"T")

### Planteamiento de necesidades comunidad

In [6]:
needs = pd.read_excel("CaseData.xlsx", sheet_name = f"necesidades_{location['name_data']}", index_col = 0)

idx_hour_0 = needs.columns.get_loc(0)
needs_n = needs.iloc[:,:idx_hour_0]

needs_load = needs.iloc[:, idx_hour_0:].T.astype(float)
needs_load.index.name = "Hora"

visual_dfs = {"Eficiencias por necesidad" : needs_n, "Carga diaria nececidades (kW)" : needs_load}

needs_load = pd.concat([needs_load] * 365, ignore_index=True)

tabbed_df_viewer(visual_dfs)

### Parámetros del modelo

In [7]:
data_model = {}

# PARÁMETROS GEOGRÁFICOS
data_model["lat"] = location["lat"]
data_model["lon"] = location["lon"]

# CANTIDAD DE DATOS TEMPORALES
data_model["len_t"] = 8760

# PARÁMETROS ECONÓMICOS
data_model["interest"] = 0.1
data_model["lifeyears"] = 25

# PARÁMETROS NECESIDADES
data_model["needs"] = {}
data_model["needs"]["active"] = True
if data_model["needs"]["active"]:
    data_model["needs"]["n"] = needs_n
    data_model["needs"]["load"] = needs_load

# PARÁMETROS DE LA CARGA ELÉCTRICA
data_model["load_el"] = {}
data_model["load_el"]["active"] = True
if data_model["load_el"]["active"]:    
    data_model["load_el"]["type"] = "variable"
    data_model["load_el"]["value"] = series_data["LOAD_EL"].to_numpy()
    
    # PARÁMETROS DE LA ENERGIA ELÉCTRICA NO SUMINISTRADA
    data_model["load_el"]["ENS_EL"] = {}
    data_model["load_el"]["ENS_EL"]["active"] = True
    data_model["load_el"]["ENS_EL"]["type"] = "fixed"
    data_model["load_el"]["ENS_EL"]["value"] = 2

# PARÁMETROS DE LA CARGA TÉRMICA
data_model["load_th"] = {}
data_model["load_th"]["active"] = True
if data_model["load_th"]["active"]:    
    data_model["load_th"]["type"] = "variable"
    data_model["load_th"]["value"] = series_data["LOAD_TH"].to_numpy()
    
    # PARÁMETROS DE LA ENERGIA TÉRMICA NO SUMINISTRADA
    #data_model["load_th"]["ENS_TH"] = {}
    #data_model["load_th"]["ENS_TH"]["active"] = True
    #data_model["load_th"]["ENS_TH"]["type"] = "fixed"
    #data_model["load_th"]["ENS_TH"]["value"] = 5

# PARÁMETROS DE LA CARGA DE REFRIGERACIÓN
data_model["load_cl"] = {}
data_model["load_cl"]["active"] = True
if data_model["load_cl"]["active"]:    
    data_model["load_cl"]["type"] = "variable"
    data_model["load_cl"]["value"] = series_data["LOAD_CL"].to_numpy()
    
    # PARÁMETROS DE LA ENERGIA DE REFRIGERACIÓN NO SUMINISTRADA
    #data_model["load_cl"]["ENS_CL"] = {}
    #data_model["load_cl"]["ENS_CL"]["active"] = True
    #data_model["load_cl"]["ENS_CL"]["type"] = "fixed"
    #data_model["load_cl"]["ENS_CL"]["value"] = 3
   


# PARÁMETROS DE LOS INVERSORES
data_model["inverters"] = {}
data_model["inverters"]["active"] = True
if data_model["inverters"]["active"]:
    data_model["inverters"]["type"] = catalogo["Hybrid OnGrid"]
    data_model["inverters"]["flex"] = False


# PARÁMETROS DE LAS BATERÍAS
data_model["batteries"] = {}
data_model["batteries"]["active"] = True
if data_model["batteries"]["active"]:
    data_model["batteries"]["type"] = catalogo["BattModules"]
    data_model["batteries"]["soc_0"] = 0.5

# PARÁMETROS DE LOS PANELES SOLARES
data_model["pv_modules"] = {}
data_model["pv_modules"]["active"] = True
if data_model["pv_modules"]["active"]:

    data_model["pv_modules"]["type"] = catalogo["PVModules"]

    Temp_min = extract_tem_min(data_model["lat"], data_model["lon"])
    for k in data_model["pv_modules"]["type"].columns:
        data_model["pv_modules"]["type"].loc['Voc_max', k] = np.round(
            data_model["pv_modules"]["type"].loc['Voc_STC',k]*(1+(data_model["pv_modules"]["type"].loc['Tc_Voc',k]/100)*(Temp_min-25))
        ,2)

# PARÁMETROS DE LAS TURBINAS EÓLICAS
data_model["windgen"] = {}
data_model["windgen"]["active"] = True
if data_model["windgen"]["active"]:
    data_model["windgen"]["type"] = catalogo["WindTurbines"]

# PARÁMETROS LAS CALDERAS (BOILERS)
data_model["boilers"] = {}
data_model["boilers"]["active"] = True
if data_model["boilers"]["active"]:
    data_model["boilers"]["type"] = catalogo["Boilers"]

# PARÁMETROS LOS CHPs
data_model["chps"] = {}
data_model["chps"]["active"] = True
if data_model["chps"]["active"]:
    data_model["chps"]["type"] = catalogo["CHPs"]

# PARÁMETROS LOS ENFRIADORES DE ABSORCIÓN
data_model["abs_chillers"] = {}
data_model["abs_chillers"]["active"] = True
if data_model["abs_chillers"]["active"]:
    data_model["abs_chillers"]["type"] = catalogo["AbsorptionChillers"]

# PARÁMETROS LOS ENFRIADORES ELÉCTRICOS
data_model["el_chillers"] = {}
data_model["el_chillers"]["active"] = True
if data_model["el_chillers"]["active"]:
    data_model["el_chillers"]["type"] = catalogo["ElectricChillers"]

# PARÁMETROS LOS CALENTADORES ELÉCTRICOS
data_model["el_heaters"] = {}
data_model["el_heaters"]["active"] = True
if data_model["el_heaters"]["active"]:
    data_model["el_heaters"]["type"] = catalogo["ElectricHeaters"]

# PARÁMETROS LOS CALENTADORES ELÉCTRICOS
data_model["pv_thermal"] = {}
data_model["pv_thermal"]["active"] = True
if data_model["pv_thermal"]["active"]:
    data_model["pv_thermal"]["type"] = catalogo["PVThermal"]

# PARÁMETROS DE LA RED ELÉCTRICA
data_model["grid_el"] = {}
data_model["grid_el"]["active"] = True
if data_model["grid_el"]["active"]:
    data_model["grid_el"]["pmax_buy"] = 800
    data_model["grid_el"]["pmax_sell"] = 0

    data_model["grid_el"]["buy_price"] = {}
    data_model["grid_el"]["buy_price"]["type"] = "variable"
    data_model["grid_el"]["buy_price"]["value"] = series_data["C_BUY_EL"].to_numpy()

    data_model["grid_el"]["sell_price"] = {}
    data_model["grid_el"]["sell_price"]["type"] = "variable"
    data_model["grid_el"]["sell_price"]["value"] = series_data["C_SELL_EL"].to_numpy()

    data_model["grid_el"]["av"] = {}
    data_model["grid_el"]["av"]["active"] = True

    if data_model["grid_el"]["av"]["active"]:
        data_model["grid_el"]["av"]["value"] = series_data["AV_GRID_EL"].to_numpy()

# PARÁMETROS DE LA RED DE GAS
data_model["grid_gas"] = {}
data_model["grid_gas"]["active"] = True
if data_model["grid_gas"]["active"]:
    
    data_model["grid_gas"]["pmax_buy"] = 300
    
    data_model["grid_gas"]["buy_price"] = {}
    data_model["grid_gas"]["buy_price"]["type"] = "variable"
    data_model["grid_gas"]["buy_price"]["value"] = series_data["C_BUY_GAS"].to_numpy()

    data_model["grid_gas"]["av"] = {}
    data_model["grid_gas"]["av"]["active"] = True

    if data_model["grid_gas"]["av"]["active"]:
        data_model["grid_gas"]["av"]["value"] = series_data["AV_GRID_GAS"].to_numpy()



# PARÁMETROS DEL GENERADOR DIESEL
#https://cdn.ade-power.com/assets/pdf/generators/cat/cat-de200gc-data-sheet.pdf
data_model["generator"] = {}
data_model["generator"]["active"] = False
if data_model["generator"]["active"]:
    data_model["generator"]["fuel_cost"] = 0.59
    data_model["generator"]["gen_cost"] = 0
    data_model["generator"]["pmax"] = 175
    data_model["generator"]["fmin"] = 1.45
    data_model["generator"]["fmax"] = 50.8 
    data_model["generator"]["fm"] = 0.282
    data_model["generator"]["gen_OM_cost"] = 3
    data_model["generator"]["min_p_load"] = 20

    data_model["generator"]["av"] = {}
    data_model["generator"]["av"]["active"] = True
    if data_model["generator"]["av"]["active"]:
        data_model["generator"]["av"]["value"] = series_data["AV_DIESEL"].to_numpy()

# PARÁMETROS DEL ÁREA DISPONIBLE
data_model["area"] = {}
data_model["area"]["active"] = False
if data_model["area"]["active"]:
    data_model["area"]["value"] = 20


# PARÁMETROS LÍMITE DE INVERSIÓN
data_model["max_invest"] = {}
data_model["max_invest"]["active"] = False  
#data_model["max_invest"]["value"] = 

# PARÁMETROS DE BONOS DE CARBONO
data_model["environment"] = {}
data_model["environment"]["active"] = True
data_model["environment"]["mu"] = 266.76
data_model["environment"]["Cbono"] = 4.23

## Resolver modelo esocástico

### Crear Maestro optimizacion

In [8]:
maestro_opt_estocastico = Maestro_Optimizacion(
    data_model = data_model,
    location = location,
    df_meteo = df_meteo,
    info_meteo = info,
    estocastico = True  
)

print("Escenarios:", maestro_opt_estocastico.escenarios)


Escenarios: ['bad_bad', 'bad_mean', 'bad_good', 'mean_bad', 'mean_mean', 'mean_good', 'good_bad', 'good_mean', 'good_good']


### Resolver modelo estocástico

In [9]:
maestro_opt_estocastico.LS_optimizacion()

[   19.58] Initializing SPBase
Current Iteration: 1 Time Elapsed:    0.00 Current Objective: -Inf
Current Iteration: 2 Time Elapsed: 2102.21 Time Spent on Last Master:    0.36 Time Spent Generating Last Cut Set: 2101.84 Current Objective:    0.00
Current Iteration: 3 Time Elapsed: 4243.01 Time Spent on Last Master:    0.49 Time Spent Generating Last Cut Set: 2140.30 Current Objective: 925106.48
Current Iteration: 4 Time Elapsed: 6595.94 Time Spent on Last Master:    0.48 Time Spent Generating Last Cut Set: 2352.43 Current Objective: 4173157.43
Current Iteration: 5 Time Elapsed: 8246.99 Time Spent on Last Master:    0.52 Time Spent Generating Last Cut Set: 1650.51 Current Objective: 5885205.01
Current Iteration: 6 Time Elapsed: 10653.30 Time Spent on Last Master:    0.48 Time Spent Generating Last Cut Set: 2405.81 Current Objective: 6382020.20
Current Iteration: 7 Time Elapsed: 12799.72 Time Spent on Last Master:    0.53 Time Spent Generating Last Cut Set: 2145.88 Current Objective: 644

In [10]:
variables = maestro_opt_estocastico.ls.gather_var_values_to_rank0()
for ((scen_name, var_name), var_value) in variables.items():
    print(scen_name, var_name, var_value)

bad_bad X_PV[EcoGreen-540W,HYD-20kTL] 0.0
bad_bad X_PV[Risen-590W,HYD-20kTL] 2491.0
bad_bad X_PVs[EcoGreen-540W,HYD-20kTL] 0.0
bad_bad X_PVs[Risen-590W,HYD-20kTL] 109.0
bad_bad X_PT['Type 1 -2.8m2'] 0.0
bad_bad X_B['BYD-22.1',HYD-20kTL] 0.0
bad_bad X_B[PylonTech-UP5000,HYD-20kTL] 152.0
bad_bad X_Bs['BYD-22.1',HYD-20kTL] 0.0
bad_bad X_Bs[PylonTech-UP5000,HYD-20kTL] 19.0
bad_bad X_CH[EcoGreen-540W,'BYD-22.1',HYD-20kTL] 0.0
bad_bad X_CH[EcoGreen-540W,PylonTech-UP5000,HYD-20kTL] 0.0
bad_bad X_CH[Risen-590W,'BYD-22.1',HYD-20kTL] 0.0
bad_bad X_CH[Risen-590W,PylonTech-UP5000,HYD-20kTL] 49.0
bad_bad Y_CH[EcoGreen-540W,'BYD-22.1',HYD-20kTL] 0.0
bad_bad Y_CH[EcoGreen-540W,PylonTech-UP5000,HYD-20kTL] 0.0
bad_bad Y_CH[Risen-590W,'BYD-22.1',HYD-20kTL] 0.0
bad_bad Y_CH[Risen-590W,PylonTech-UP5000,HYD-20kTL] 1.0
bad_bad X_WT[FX-20kW] 0.0
bad_bad X_WT[TUGE-10kW] 0.0
bad_bad X_BOI[Type 1 - 10kWth] 13.0
bad_bad X_BOI[Type 1 - 5kWth] 1.0
bad_bad X_EH[Type 1 - 10kW] 9.0
bad_bad X_EH[Type 2 - 15kW] 3.0
bad

In [23]:
from pyomo.environ import value

VPN_F = np.sum([round(1/np.power(1+data_model["interest"],i),3) for i in np.arange(1,data_model["lifeyears"]+1)])
Init_invest = value(ls.local_scenarios[ls.local_scenario_names[0]].FirstStage)
LCOE = 0
CPN = Init_invest

for i in ls.local_scenarios.keys():
    m = ls.local_scenarios[i]
    CPN += value(m.SecondStage)*m._mpisppy_probability
    LCOE += ((value(m.FirstStage) + value(m.SecondStage))/(sum(m.Carga[t] - m.ENS_EL[t].value + sum(m.PpvG[tch,t].value for tch in m.CH) + m.PTG[t].value for t in m.T)*VPN_F))*m._mpisppy_probability


print(f'Inversión incial: {Init_invest}, CPN: {CPN}, LCOE: {LCOE}')

Inversión incial: 1431615.9, CPN: 1772563.9984243372, LCOE: 0.2504404804781522


In [25]:
ls.tree_solution_available=True

In [26]:
ls.write_tree_solution(f'{abrv_name}_est_res_off')

### Resolver modelo deterministico

In [24]:
abrv_name = location["name_data"]

In [ ]:
if data_model["pv_modules"]["active"]:
    data_model["pv_modules"]["Pmpp"] = power_PV_calculation(df_meteo, data_model["pv_modules"]["type"], 0, 10, data_model["lat"], type="electric") 

if data_model["pv_thermal"]["active"]:
    data_model["pv_thermal"]["Pmpp"] = power_PV_calculation(df_meteo, data_model["pv_thermal"]["type"], 0, 10, data_model["lat"], type="thermal") 

if data_model["windgen"]["active"]:
    Profiles, Wind_generation = calculate_WT_power(df_meteo, data_model["windgen"]["type"], 0.001, 20, info['Elevation'].iloc[0])               
    data_model["windgen"]["generation"] = Wind_generation 

In [26]:
model = create_model(data_model)

In [28]:
VPN_F = [round(1/np.power(1+data_model["interest"],i),3) for i in np.arange(1,data_model["lifeyears"]+1)]
VPN_FS = round(np.sum(VPN_F),3)   

valor_base = VPN_FS*sum(
    (model.cost_ens_el[t]*(1-model.grid_el_av[t]) + model.grid_el_av[t]*model.price_buy_grid_el[t])*(model.load_el[t] + sum(model.needs_load[t,nds] for nds in model.needs))     
for t in model.T)

print(valor_base)

10688268.1267291


In [ ]:
def scenario_creator(scenario_name):
    
    model = create_model(data_model)
    vars_first_stage = [
        model.X_PV,
        model.X_PVs,
        model.X_PT,
        model.X_B,
        model.X_Bs,
        model.X_CH,
        model.Y_CH,
        model.X_WT,
        model.X_BOI,
        model.X_EH,
        model.X_CHP,
        model.X_AC,
        model.X_EC,
        model.Y_NEEDS
    ]
    sputils.attach_root_node(model, model.FirstStage, vars_first_stage)
    model._mpisppy_probability = 1
    return model

In [30]:
escenarios = ["main"]

In [ ]:
bounds = {name: -3e15 for name in escenarios}
options = {
    "root_solver": "cplex",
    "sp_solver": "cplex",
    "sp_solver_options" : {"threads" : 12},
    "max_iter": 300,
    "valid_eta_lb": bounds
}

ls = LShapedMethod(options, escenarios, scenario_creator)
result = ls.lshaped_algorithm()

[ 1509.82] Initializing SPBase
Current Iteration: 1 Time Elapsed:    0.00 Current Objective: -Inf
Current Iteration: 2 Time Elapsed:  242.82 Time Spent on Last Master:    0.25 Time Spent Generating Last Cut Set:  242.57 Current Objective: -3000000000000000.00
Current Iteration: 3 Time Elapsed:  396.74 Time Spent on Last Master:    0.23 Time Spent Generating Last Cut Set:  153.68 Current Objective:    0.00
Current Iteration: 4 Time Elapsed:  529.88 Time Spent on Last Master:    0.22 Time Spent Generating Last Cut Set:  132.92 Current Objective: 252655.50
Current Iteration: 5 Time Elapsed:  689.22 Time Spent on Last Master:    0.23 Time Spent Generating Last Cut Set:  159.11 Current Objective: 3879562.58
Current Iteration: 6 Time Elapsed:  851.17 Time Spent on Last Master:    0.24 Time Spent Generating Last Cut Set:  161.71 Current Objective: 5342176.34
Current Iteration: 7 Time Elapsed: 1038.09 Time Spent on Last Master:    0.23 Time Spent Generating Last Cut Set:  186.69 Current Object

In [ ]:



results = LS_Generador_Resultados(ls)

results.resultado_valor_inversion_inicial()

#variables = ls.gather_var_values_to_rank0()
#for ((scen_name, var_name), var_value) in variables.items():
#    print(scen_name, var_name, var_value)    

Valor inversión inicial:  1305440.7999999998


1305440.7999999998

In [48]:
variables = ls.gather_var_values_to_rank0()
for ((scen_name, var_name), var_value) in variables.items():
    print(scen_name, var_name, var_value)

main X_PV[EcoGreen-540W,HYD-20kTL] 0.0
main X_PV[Risen-590W,HYD-20kTL] 2440.0
main X_PT['Type 1 -2.8m2'] 0.0
main X_B['BYD-22.1',HYD-20kTL] 0.0
main X_B[PylonTech-UP5000,HYD-20kTL] 152.0
main X_Bs['BYD-22.1',HYD-20kTL] 0.0
main X_Bs[PylonTech-UP5000,HYD-20kTL] 19.0
main X_CH[EcoGreen-540W,'BYD-22.1',HYD-20kTL] 0.0
main X_CH[EcoGreen-540W,PylonTech-UP5000,HYD-20kTL] 0.0
main X_CH[Risen-590W,'BYD-22.1',HYD-20kTL] 0.0
main X_CH[Risen-590W,PylonTech-UP5000,HYD-20kTL] 48.0
main Y_CH[EcoGreen-540W,'BYD-22.1',HYD-20kTL] 0.0
main Y_CH[EcoGreen-540W,PylonTech-UP5000,HYD-20kTL] 0.0
main Y_CH[Risen-590W,'BYD-22.1',HYD-20kTL] 0.0
main Y_CH[Risen-590W,PylonTech-UP5000,HYD-20kTL] 1.0
main X_WT[FX-20kW] 0.0
main X_WT[TUGE-10kW] 0.0
main X_BOI[Type 1 - 10kWth] 13.0
main X_BOI[Type 1 - 5kWth] 1.0
main X_EH[Type 1 - 10kW] 12.0
main X_EH[Type 2 - 15kW] 1.0
main X_CHP[Type 1 - 5kW] 0.0
main X_CHP[Type 2 - 10kW] 0.0
main X_AC[Type 1 - 10kWcl] 0.0
main X_AC[Type 1 - 5kWcl] 0.0
main X_EC[Type 1 - 8kWcl] 4.0


In [37]:
Init_invest = value(ls.local_scenarios[ls.local_scenario_names[0]].FirstStage)

print(Init_invest)

1986117.9079999998


In [ ]:
from pyomo.environ import value

VPN_F = np.sum([round(1/np.power(1+data_model["interest"],i),3) for i in np.arange(1,data_model["lifeyears"]+1)])
Init_invest = value(ls.local_scenarios[ls.local_scenario_names[0]].FirstStage)
LCOE = 0
CPN = Init_invest

for i in ls.local_scenarios.keys():
    m = ls.local_scenarios[i]
    CPN += value(m.SecondStage)*m._mpisppy_probability
    LCOE += ((value(m.FirstStage) + value(m.SecondStage))/(sum(m.Carga[t] - m.ENS_EL[t].value + sum(m.PpvG[tch,t].value for tch in m.CH) + m.PTG[t].value for t in m.T)*VPN_F))*m._mpisppy_probability


print(f'Inversión incial: {Init_invest}, CPN: {CPN}, LCOE: {LCOE}')

Inversión incial: 1460268.5799999998, CPN: 1655434.1866481751, LCOE: 0.23334776049219885


In [15]:
ls.local_scenarios.keys()

dict_keys(['main'])

In [39]:
dir(ls)

['E1_tolerance',
 '_LShaped_bound',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slotnames__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_root_etas',
 '_assign_bundles',
 '_attach_nlens',
 '_attach_nonant_indices',
 '_attach_varid_to_nonant_index',
 '_calculate_scenario_ranks',
 '_check_nodenames',
 '_check_variable_probabilities_sum',
 '_compute_unconditional_node_probabilities',
 '_create_communicators',
 '_create_root_no_scenarios',
 '_create_root_with_scenarios',
 '_create_scenarios',
 '_create_shadow_root',
 '_look_and_leap',
 '_options_check',
 '_rank_slices',
 '_scenario_slices',
 '_scenario_tree',
 '_set_sense',
 '_spcomm',
 '_use_variable_probability_setter',
 '_verify_nonant_lengths',
 'a

In [42]:
ls.local_scenarios

{'main': <pyomo.core.base.PyomoModel.ConcreteModel at 0x22f23c92400>}

In [43]:
import pickle

# Guardar el modelo optimizado
with open("modelo_optimizado.pkl", "wb") as archivo:
    pickle.dump(ls.local_scenarios, archivo)

print("Modelo guardado en modelo_optimizado.pkl")

AttributeError: Can't pickle local object 'create_model.<locals>.balance_power_needs'